## deploy

In [34]:
import pandas as pd
import re
import os

In [35]:
path_dir = r'C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks'
src_dir = os.path.join(path_dir,'txt files')
dst_tmp_dir = os.path.join(path_dir,'csv files temp')
dst_dir = os.path.join(path_dir,'csv files')

lst_files = os.listdir(src_dir)

In [36]:
iter_tp = 0
for iter_fl in lst_files:
    print(iter_tp,iter_fl)
    iter_tp+=1

0 dialog-babi-candidates.txt
1 dialog-babi-kb-all.txt
2 dialog-babi-task1-API-calls-dev.txt
3 dialog-babi-task1-API-calls-trn.txt
4 dialog-babi-task1-API-calls-tst-OOV.txt
5 dialog-babi-task1-API-calls-tst.txt
6 dialog-babi-task2-API-refine-dev.txt
7 dialog-babi-task2-API-refine-trn.txt
8 dialog-babi-task2-API-refine-tst-OOV.txt
9 dialog-babi-task2-API-refine-tst.txt
10 dialog-babi-task3-options-dev.txt
11 dialog-babi-task3-options-trn.txt
12 dialog-babi-task3-options-tst-OOV.txt
13 dialog-babi-task3-options-tst.txt
14 dialog-babi-task4-phone-address-dev.txt
15 dialog-babi-task4-phone-address-trn.txt
16 dialog-babi-task4-phone-address-tst-OOV.txt
17 dialog-babi-task4-phone-address-tst.txt
18 dialog-babi-task5-full-dialogs-dev.txt
19 dialog-babi-task5-full-dialogs-trn.txt
20 dialog-babi-task5-full-dialogs-tst-OOV.txt
21 dialog-babi-task5-full-dialogs-tst.txt
22 dialog-babi-task6-dstc2-candidates.txt
23 dialog-babi-task6-dstc2-dev.txt
24 dialog-babi-task6-dstc2-kb.txt
25 dialog-babi-task

## task 1,2

In [79]:
fl_no = 2
path_final = os.path.join(src_dir,lst_files[fl_no])
print(path_final)

C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks\txt files\dialog-babi-task1-API-calls-dev.txt


In [80]:
df = pd.DataFrame([])
path_to_use = path_final
ct = 0
with open(path_to_use) as topo_file:
    cnt = 0
    cnt_sub = 0
    ar_3,ar_4 = [],[]
    ar_1,ar_2 = [],[]

    for line in topo_file:
        # if ct>120: break
        # ct+=1

        line = line.strip()    
        if not line.strip(): continue

        init_num = re.findall(r'\d+',line)        

        if init_num[0]=='1':

            ar_5 = cnt
            cnt+=1

            if len(ar_3)==0: continue
            
            data = list(
                zip(                 
                    ar_3,
                    ar_4,
                    [ar_5]*len(ar_3)       
                    ))

            df = df.append(data,ignore_index=True)
            cnt_sub = 0
            ar_3,ar_4 = [],[]
            continue

        line_splits = line[2:].split('\t')
        ar_3.append(line_splits[0])
        ar_4.append(line_splits[1])
        cnt_sub+=1

df = df.rename(columns={
     0: 'Call',
     1: 'Response',
     2: 'Count'
     })
print(df.shape)
fl_dst = lst_files[fl_no][:-3] + 'csv'
path_dst_tmp = os.path.join(dst_tmp_dir,fl_dst)
df.to_csv(path_dst_tmp,index=False)

(5009, 3)


In [81]:
df.head()

,Call,Response,Count
0,can you book a table for six people with frenc...,i'm on it,1
1,<SILENCE>,where should it be,1
2,in bombay,which price range are looking for,1
3,i am looking for a cheap restaurant,ok let me look into some options for you,1
4,<SILENCE>,api_call french bombay six cheap,1


In [82]:
prev_count = 0
txt = []
for i in range(len(df)):    
    if df.iloc[i][2]!=prev_count:
        prev_count = df.iloc[i][2]
        txt.append('')
        continue
    txt.append('<context>')  
df['TAG'] = txt

In [86]:
TAGS = [

    '<profile> ',
    ' <context>',
    ' <user> ',
    ' <system> ',
    '<query> ']

c = 0
mn_ar_1 = []
mn_ar_2 = []
mn_ar_3 = []
prev_count = 0
prev_string = ''
for i in range(len(df)):    
    #if c > 100: break
    c+=1    

    if df.iloc[i][2]!=prev_count:
        prev_string = ''
        prev_count = df.iloc[i][2]
    
    a2 = df.iloc[i][0]
    a3 = df.iloc[i][1]    
    if prev_string=='':
            call_string = prev_string + TAGS[4]+ a2    
    else:
        call_string = prev_string +' '+TAGS[4]+ a2  
    prev_string = prev_string + TAGS[2]+  a2 + ' ' + TAGS[3] + a3
    mn_ar_1.append(call_string)
    mn_ar_2.append(a3)

ndf = pd.DataFrame(            
            list(zip(
                mn_ar_1,mn_ar_2                  
                )),

            columns =[
                'Call',
                'Response'
                ]
                ) 
ndf['Call'] = df['TAG'] + ndf['Call']


print(ndf.shape)
fl_dst = lst_files[fl_no][:-3] + '.csv'
path_dst = os.path.join(dst_dir,fl_dst)
ndf.to_csv(path_dst,index=False)
    

(5009, 2)


In [73]:
ndf.head(10)

,Call,Response
0,<query> can you book a table for six people w...,i'm on it
1,<context> <user> can you book a table for six ...,where should it be
2,<context> <user> can you book a table for six ...,which price range are looking for
3,<context> <user> can you book a table for six ...,ok let me look into some options for you
4,<context> <user> can you book a table for six ...,api_call french bombay six cheap
5,<query> can you make a restaurant reservation...,i'm on it
6,<context> <user> can you make a restaurant res...,where should it be
7,<context> <user> can you make a restaurant res...,ok let me look into some options for you
8,<context> <user> can you make a restaurant res...,api_call italian rome six cheap
9,<query> may i have a table with british cuisi...,i'm on it


(5009, 2)


## loop define

In [99]:
def convert_to_df(path_final):
    df = pd.DataFrame([])
    path_to_use = path_final
    ct = 0
    with open(path_to_use) as topo_file:
        cnt = 0
        cnt_sub = 0
        ar_3,ar_4 = [],[]
        ar_1,ar_2 = [],[]

        for line in topo_file:
            # if ct>120: break
            # ct+=1

            line = line.strip()    
            if not line.strip(): continue

            init_num = re.findall(r'\d+',line)        

            if init_num[0]=='1':

                ar_5 = cnt
                cnt+=1

                if len(ar_3)==0: continue
                
                data = list(
                    zip(                 
                        ar_3,
                        ar_4,
                        [ar_5]*len(ar_3)       
                        ))

                df = df.append(data,ignore_index=True)
                cnt_sub = 0
                ar_3,ar_4 = [],[]
                continue

            line_splits = line[2:].split('\t')
            ar_3.append(line_splits[0])
            ar_4.append(line_splits[1])
            cnt_sub+=1

    df = df.rename(columns={
        0: 'Call',
        1: 'Response',
        2: 'Count'
        })
    print(df.shape)
    fl_dst = lst_files[fl_no][:-3] + 'csv'
    path_dst_tmp = os.path.join(dst_tmp_dir,fl_dst)
    df.to_csv(path_dst_tmp,index=False)
    return df

TAGS = [

    '<profile> ',
    ' <context>',
    ' <user> ',
    ' <system> ',
    '<query> ']

def add_context(df):
    prev_count = 0
    txt = []
    for i in range(len(df)):    
        if df.iloc[i][2]!=prev_count:
            prev_count = df.iloc[i][2]
            txt.append('')
            continue
        txt.append('<context>')  
    df['TAG'] = txt
    return df



def parse_from_df(df):
    c = 0
    mn_ar_1 = []
    mn_ar_2 = []
    mn_ar_3 = []
    prev_count = 0
    prev_string = ''
    for i in range(len(df)):    
        #if c > 100: break
        c+=1    

        if df.iloc[i][2]!=prev_count:
            prev_string = ''
            prev_count = df.iloc[i][2]
        
        a2 = df.iloc[i][0]
        a3 = df.iloc[i][1]            
        if prev_string=='':
            call_string = prev_string + TAGS[4]+ a2    
        else:
            call_string = prev_string +' '+TAGS[4]+ a2    
        prev_string = prev_string + TAGS[2]+ a2  + TAGS[3] + a3
        mn_ar_1.append(call_string)
        mn_ar_2.append(a3)

    ndf = pd.DataFrame(            
                list(zip(
                    mn_ar_1,mn_ar_2                  
                    )),

                columns =[
                    'Call',
                    'Response'
                    ]
                    ) 

    ndf['Call'] = df['TAG'] + ndf['Call']
    print(ndf.shape)
    fl_dst = lst_files[fl_no][:-3] + '.csv'
    path_dst = os.path.join(dst_dir,fl_dst)
    ndf.to_csv(path_dst,index=False)   

In [91]:
for i in range(2,10):
    fl_no = i
    path_final = os.path.join(src_dir,lst_files[fl_no])
    print(path_final)

C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks\txt files\dialog-babi-task1-API-calls-dev.txt
C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks\txt files\dialog-babi-task1-API-calls-trn.txt
C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks\txt files\dialog-babi-task1-API-calls-tst-OOV.txt
C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks\txt files\dialog-babi-task1-API-calls-tst.txt
C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks\txt files\dialog-babi-task2-API-refine-dev.txt
C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks\txt files\dialog-babi-task2-API-refine-trn.txt
C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks\txt files\dialog-babi-task2-API-refine-tst-OOV.txt
C:\Users\ydjoe\Documents\nlp\dialog-bAbI-tasks\txt files\dialog-babi-task2-API-refine-tst.txt


## loop run

In [100]:
for i in range(2,10):
    fl_no = i
    path_final = os.path.join(src_dir,lst_files[fl_no])
    df = convert_to_df(path_final)
    df = add_context(df)
    parse_from_df(df)
    

(5009, 3)
(5009, 2)
(5020, 3)
(5020, 2)
(5017, 3)
(5017, 2)
(4930, 3)
(4930, 2)
(8505, 3)
(8505, 2)
(8489, 3)
(8489, 2)
(8453, 3)
(8453, 2)
(8456, 3)
(8456, 2)
